### Read all FHIR SD and find all binding strengths

- load package data to inspect
- read SDs
- print binding by strength


In [1]:
from pathlib import Path
from json import load, dumps, loads
from IPython.display import display, Markdown, HTML 

### get package and create obj and find all references types


In [17]:
us_core_311 = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1') / 'package'
us_core_320 = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.2.0') / 'package'


def get_320(f,id):
    print(f, id)
    f320 = us_core_320 / f.name
    obj320 = loads(f320.read_text())
    # print(f'obj320 = {obj320["id"]}')
    try:
      element320 = [element for element in obj320['differential']['element'] if element['id'] == id][0]
    except IndexError:
        return
    # print(f'element320 = {element320["id"]}')
    my_type = [t for t in element320['type'] if t['code'] == 'Reference'][0]
    # print(f'my_type = {my_type}')
    targets = [t for t in my_type['targetProfile']]
    print(f'all_targets = {targets}')
    return targets


i = 1
for f in us_core_311.glob('StructureDefinition*.json'):
  print(f)
  obj = loads(f.read_text())
  print(f"{i}. {obj['id']}")
  for element in obj['differential']['element']:
      try:
          for my_type in element['type']:
              if my_type['code'] == 'Reference':
                  print(f''' - Reference element: {element['path']}''')
                  ms_targets = [t for t in my_type['targetProfile']]
                  print(f''' - 3.1.1 Reference element MS targets: {ms_targets}''')
                  all_targets = get_320(f,element['id'])
                  try:
                    optional_targets = [item for item in all_targets if item not in ms_targets]
                  except TypeError:
                     optional_targets = []
                     
                  print(f''' - 3.2.0 Reference element addition targets targets: {optional_targets}''')
      except KeyError:
          # print(f"no type in {element['path']}")
          pass
  i+=1

/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/StructureDefinition-us-core-observation-lab.json
1. us-core-observation-lab
 - Reference element: Observation.subject
 - 3.1.1 Reference element MS targets: ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/StructureDefinition-us-core-observation-lab.json Observation.subject
all_targets = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']
 - 3.2.0 Reference element addition targets targets: []
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/StructureDefinition-us-core-careteam.json
2. us-core-careteam
 - Reference element: CareTeam.subject
 - 3.1.1 Reference element MS targets: ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']
/Users/ehaas/.fhir/packages/hl7.fhir.us.core#3.1.1/package/StructureDefinition-us-core-careteam.json CareTeam.subject
all_targets = ['http://hl7.org/fhir/us/core/StructureDefinition/u

TypeError: 'NoneType' object is not iterable

### Get SDs

In [79]:
def inherited(strength,element_path,sd_type):
    print(strength,element_path,sd_type)
    for f in fhir_path.iterdir():
        if str(f.name)==f'StructureDefinition-{sd_type}.json':
            print(f'### {f.name}')
            sd_dict=loads(f.read_text())
            for element in sd_dict['differential']['element']:
                if element.get('path')==element_path:
                    try:
                        base_strength=element['binding'].get('strength','no_strength')  
                    except KeyError:
                        base_strength="no binding!"
                    print(base_strength)
    try:
        return base_strength==strength
    except UnboundLocalError:
        return False

In [ ]:
for f in uscore_path.iterdir():
    if str(f.name).startswith('StructureDefinition'):
        #print(f'### {f.name}')
        sd_dict=loads(f.read_text())
        #print(sd_dict['type'])
        for element in sd_dict['differential']['element']:
            if element.get('binding'):
                vs = element['binding'].get('valueSet','No ValueSet')
                vs = vs.split('|')[0] #strip version
                strength = element['binding'].get('strength','no_strength')
                print('foo')
                is_inherited = inherited(strength=strength,element_path=element["path"],sd_type=sd_dict['type'])
                bindings[strength]= f'{bindings[strength]}\n|{sd_dict["title"]}|{element["path"]}|{vs}|{f"**{is_inherited}**" if is_inherited == False else is_inherited}|'

In [ ]:
for strength,binding in bindings.items():
    print(f'### {strength.upper()}')
    print(bindings[strength])
    #display(Markdown(f'### {strength.upper()}'))
    #display(Markdown(bindings[strength]))   